In [1]:
import os
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# Set directory path to current directory
cv_path = os.path.abspath('../control-variables')
dir_path = os.path.abspath('')
 
cv = os.path.join(cv_path, 'inde-variable-data.xlsx')
dat = os.path.join(dir_path, 'results_v1.xlsx')
control_variable = os.path.join(dir_path, 'control_variables.xlsx')
cpi = os.path.join(dir_path, 'cpi-data.xlsx')
trade = os.path.join(dir_path, 'test_data.xlsx')

%store -r countryList

Trade openning for countries

In [126]:
# Import CPI data into dataframe, skip the first three rows and use defined columns

trade_df = pd.read_excel(trade, sheet_name='Sheet1', usecols="A, AU:BL", skiprows=[0,1,2])

# Fill nan values with 0
trade_df.fillna(0, inplace=True)

# Drop countries that are not in the defined country list
trade_df = trade_df[trade_df['Country Name'].isin(countryList)]

trade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 4 to 265
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  39 non-null     object 
 1   2002          39 non-null     float64
 2   2003          39 non-null     float64
 3   2004          39 non-null     float64
 4   2005          39 non-null     float64
 5   2006          39 non-null     float64
 6   2007          39 non-null     float64
 7   2008          39 non-null     float64
 8   2009          39 non-null     float64
 9   2010          39 non-null     float64
 10  2011          39 non-null     float64
 11  2012          39 non-null     float64
 12  2013          39 non-null     float64
 13  2014          39 non-null     float64
 14  2015          39 non-null     float64
 15  2016          39 non-null     float64
 16  2017          39 non-null     float64
 17  2018          39 non-null     float64
 18  2019          39 non-null     f

In [127]:
trade_df = pd.DataFrame(trade_df).set_index('Country Name').rename_axis('year', axis=1).stack()

# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    trade_df.to_excel(writer, sheet_name='trade')


In [130]:
arranged_trade = pd.read_excel(control_variable, index_col=0, sheet_name='trade')
arranged_trade.index = pd.Series(arranged_trade.index).fillna(method='ffill')
arranged_trade = arranged_trade.sort_values(['country', 'year'], ascending=True)
arranged_trade.head()


,year,0
country,,
Angola,2002,105.301743
Angola,2003,103.901227
Angola,2004,103.579924
Angola,2005,106.590962
Angola,2006,94.625159


In [ ]:
# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    arranged_trade.to_excel(writer, sheet_name='trade-arranged')

In [6]:
# Getting the percentage change of trade % of GDP
dtrade = pd.read_excel(control_variable, index_col=0, sheet_name='trade-arranged')

dtrade['trade'] = dtrade['trade']/(dtrade['trade'].groupby('country').first())

# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    dtrade.to_excel(writer, sheet_name='dtrade')

Getting the CPI data

In [30]:
# Import CPI data into dataframe, skip the first three rows and use defined columns

cpi_df = pd.read_excel(cpi, sheet_name='data', usecols="A, AU:BL", skiprows=[0,1,2])

# Fill nan values with 0
cpi_df.fillna(0, inplace=True)

# Drop countries that are not in the defined country list
cpi_df = cpi_df[cpi_df['Country Name'].isin(countryList)]

cpi_df.head()


,Country Name,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
4,Angola,15.348130,30.423699,43.670818,53.694805,60.839012,68.292702,76.812782,87.359396,100.0,113.482468,125.146088,136.131179,146.042056,159.405448,208.335449,270.510219,323.607609,378.883721
16,Burundi,45.216066,50.030430,54.121133,61.293303,62.976062,68.273648,84.937164,93.902651,100.0,109.592166,129.495249,139.774528,145.932088,154.023568,162.583720,188.682529,183.371686,182.112340
18,Benin,79.217514,80.395671,81.098241,85.448773,88.680597,89.831731,96.970927,97.839857,100.0,102.704239,109.631314,110.101510,109.497320,109.736885,108.865519,110.791799,111.506189,110.720040
19,Burkina Faso,80.378118,82.013464,81.685222,86.925362,88.953425,88.748274,98.208661,100.770116,100.0,102.759767,106.683292,107.252702,106.975894,107.751297,108.226524,109.831523,111.979765,108.359023
33,Botswana,50.691959,55.350501,59.194982,64.291804,71.720862,76.799415,86.554622,93.502618,100.0,108.459872,116.634387,123.497863,128.934551,132.882568,136.623156,141.143034,145.713267,149.753699


In [31]:
cpi_df = pd.DataFrame(cpi_df).set_index('Country Name').rename_axis('year', axis=1).stack()
# cpi_df = cpi_df.sort_values(["Country Name", "year"], ascending=True)

print(cpi_df)

Country Name  year
Angola        2002     15.348130
              2003     30.423699
              2004     43.670818
              2005     53.694805
              2006     60.839012
                         ...    
Zimbabwe      2015    106.213145
              2016    104.573565
              2017    105.508414
              2018    116.712211
              2019    414.684309
Length: 702, dtype: float64


In [32]:
# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    cpi_df.to_excel(writer, sheet_name='cpi')


In [64]:
arranged_cpi = pd.read_excel(control_variable, index_col=0, sheet_name='cpi1')
arranged_cpi.index = pd.Series(arranged_cpi.index).fillna(method='ffill')
arranged_cpi = arranged_cpi.sort_values(['Country Name', 'year'], ascending=True)
arranged_cpi.head()


,year,0
Country Name,,
Angola,2002,15.348130
Angola,2003,30.423699
Angola,2004,43.670818
Angola,2005,53.694805
Angola,2006,60.839012


In [97]:
# gdp_df['distribution cumulative'] = gdp_df['Energy distribution'].groupby('country').cumsum()/gdp_df.groupby('country')['gdp'].last()

cpi_2010_df = pd.read_excel(control_variable, index_col=0, sheet_name="Sheet1")



In [102]:
cpi_2010_df = pd.DataFrame(cpi_2010_df)

# How to see the groupby item
# grouped_df = cpi_2010_df['cpi'].groupby('country')

# for key, item in grouped_df:
#     print(grouped_df.get_group(key), "\n\n")


cpi_2010_df['CPI 2002'] = cpi_2010_df['cpi']/(cpi_2010_df['cpi'].groupby('country').first())

# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    cpi_2010_df.to_excel(writer, sheet_name='cpi_rate')

In [34]:
# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    arranged_cpi.to_excel(writer, sheet_name='cpi_arranged')

In [4]:
# Import the GDP data into dataframe
dgdp_df = pd.read_excel(cv, index_col=0, sheet_name="dGDP")

dgdp_df = pd.DataFrame(dgdp_df.rename_axis('year', axis=1).stack())

dgdp_df = dgdp_df.sort_values(["Country Name", "year"], ascending=True)

Stacking the governance data and export to excel

In [19]:
# Import the governance data into dataframe

govern_df = pd.read_excel(cv, index_col=0, sheet_name="governance")

govern_df = pd.DataFrame(govern_df.rename_axis('year', axis=1).stack())

govern_df = govern_df.sort_values(["country", "year"], ascending=True)

In [24]:
# Write the data into an excel sheet
with pd.ExcelWriter(control_variable,
                    engine='openpyxl',
                    mode='a',
                    if_sheet_exists="new"
) as writer:
    writer.book = openpyxl.load_workbook(control_variable)
    govern_df.to_excel(writer, sheet_name='governance')


In [53]:
output = os.path.join(dir_path, 'modified_control_variables.xlsx')

# dgdp_df.to_excel(output, sheet_name='dGDP')

Calculating the accumulative sum of foreign aid flows for each sub-sectors, normalizing them by dividing them with the current year (2019)'s GDP

In [54]:
# Read the GDP data into a dataframe

gdp_df = pd.read_excel(dat, index_col=0, sheet_name="standarized data")

gdp_df = pd.DataFrame(gdp_df)

gdp_df.index = pd.Series(gdp_df.index).fillna(method='ffill')

gdp_df = gdp_df.sort_values(["country", "year"], ascending=True)

In [55]:
gdp_df['policy cumulative'] = gdp_df['Energy Policy'].groupby('country').cumsum()/gdp_df.groupby('country')['gdp'].last()/10000
gdp_df['distribution cumulative'] = gdp_df['Energy distribution'].groupby('country').cumsum()/gdp_df.groupby('country')['gdp'].last()/10000
gdp_df['nrgen cumulative'] = gdp_df['Energy generation, non-renewable sources'].groupby('country').cumsum()/gdp_df.groupby('country')['gdp'].last()/10000
gdp_df['rgen cumulative'] = gdp_df['Energy generation, renewable sources'].groupby('country').cumsum()/gdp_df.groupby('country')['gdp'].last()/10000

In [40]:
gdp_df.groupby('country')['elec_rate'].first()

country
Angola                      26.383869
Benin                       23.473083
Botswana                    31.688650
Burkina Faso                10.008075
Burundi                      3.154657
Cabo Verde                  58.600000
Cameroon                    44.066143
Central African Republic     6.226599
Chad                         3.923373
Comoros                     44.800000
Congo, Dem. Rep.             7.820706
Congo, Rep.                  0.000000
Cote d'Ivoire               51.400000
Equatorial Guinea            0.000000
Eswatini                    25.878912
Ethiopia                    26.379478
Gabon                       77.863853
Gambia, The                 31.501682
Ghana                       46.898228
Guinea                      17.940191
Guinea-Bissau                0.000000
Kenya                       18.985170
Lesotho                      3.403665
Liberia                      0.000000
Madagascar                  14.074018
Malawi                       5.558670
Mali

In [57]:
gdp_df['dElec'] = gdp_df['elec_rate']/gdp_df.groupby('country')['elec_rate'].first()

print(gdp_df['dElec'])

country
Angola      1.000000
Angola      1.041001
Angola      1.081599
Angola      1.121754
Angola      1.161564
              ...   
Zimbabwe    0.985380
Zimbabwe    1.160124
Zimbabwe    1.173809
Zimbabwe    1.187613
Zimbabwe    1.201436
Name: dElec, Length: 810, dtype: float64


In [47]:
gdp_df['elec_rate'].groupby('country').describe()

,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Angola,18.0,35.945126,6.077088,26.383869,30.984916,35.990376,40.970534,45.670315
Benin,18.0,31.870883,5.375711,23.473083,27.959788,32.045973,36.339062,40.318741
Botswana,18.0,51.210982,12.674080,31.688650,40.942740,52.510535,61.582599,70.183182
Burkina Faso,18.0,14.072480,2.704236,10.008075,11.866623,13.750000,15.904321,19.200000
Burundi,18.0,6.209023,2.624672,2.660000,4.067617,5.758337,8.052317,11.064797
Cabo Verde,18.0,78.250341,11.021653,58.600000,69.614246,80.209965,86.616959,95.533539
Cameroon,18.0,53.466654,6.011537,44.066143,48.400000,53.213651,58.114059,63.452312
Central African Republic,18.0,10.246204,2.840795,6.226599,7.621573,10.055362,12.572332,14.653953
Chad,18.0,6.991069,2.177445,3.500000,5.359232,7.050000,8.404330,10.900000


In [58]:
# Write the data into an excel sheet
with pd.ExcelWriter(dat,
                    engine='openpyxl',
                    mode='a',
                    if_sheeet_exists="overlay",
) as writer:
    writer.book = openpyxl.load_workbook(dat)
    gdp_df.to_excel(writer, sheet_name='cum_gdp_elec')

/opt/anaconda3/envs/MyPythonProject/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  """
